In [36]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import time 
from sqlalchemy import create_engine
import numpy as np

In [37]:
file_to_load = "list_of_stops.csv"
stops_data = pd.read_csv(file_to_load)
stops_data.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,False,False,False,False,True,False,"(41.857908, -87.669147)"
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,False,False,False,False,False,True,"(41.829353, -87.680622)"
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,False,False,False,False,False,False,"(41.831677, -87.625826)"


In [38]:
stops_df = stops_data[["STOP_ID", "STOP_NAME", "STATION_NAME", "STATION_DESCRIPTIVE_NAME", "MAP_ID", "Location"]]
stops_df.head()

,STOP_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,Location
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)"
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)"
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)"
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)"
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,"(41.831677, -87.625826)"


In [39]:
stops_df[["Lat","Lon"]] = stops_df.Location.str.split(",", expand=True,)


stops_df.head()

,STOP_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,Location,Lat,Lon
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)",(41.857908,-87.669147)
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,"(41.857908, -87.669147)",(41.857908,-87.669147)
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)",(41.829353,-87.680622)
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,"(41.829353, -87.680622)",(41.829353,-87.680622)
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,"(41.831677, -87.625826)",(41.831677,-87.625826)


In [40]:
stops_df = stops_df.drop(columns=['Location'])

stops_df.head()

,STOP_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,Lat,Lon
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,(41.857908,-87.669147)
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,(41.857908,-87.669147)
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,(41.829353,-87.680622)
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,(41.829353,-87.680622)
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,(41.831677,-87.625826)


In [41]:
stops_df['Lat'] = stops_df.Lat.str.replace("(", "")
stops_df['Lon'] = stops_df.Lon.str.replace(")", "")

In [42]:
stops_df.head()

,STOP_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,Lat,Lon
0,30162,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,41.857908,-87.669147
1,30161,18th (Loop-bound),18th,18th (Pink Line),40830,41.857908,-87.669147
2,30022,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,41.829353,-87.680622
3,30023,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,41.829353,-87.680622
4,30214,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,41.831677,-87.625826


In [47]:
rds_connection_string = "postgres:taylormade@localhost:5432/CTA"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [48]:
stops_df.to_sql(name='stops_table', con=engine, if_exists='append', index=False)



In [49]:
CTA = pd.read_sql_query('select * from stops_table', con=engine).head()

In [50]:
CTA.to_html("Data.html")